In [1]:
# 재시작 없이 import 파일 자동 적용
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import numpy as np
import pandas as pd
from scipy import interpolate
import neurokit2 as nk
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

In [3]:
with open("./dataset/Signal_Train.pkl", 'rb') as f:
    signal = pickle.load(f)

with open("./dataset/Target_Train.pkl", 'rb') as f:
    label = pickle.load(f)

label = label.Target.astype(int).to_list()

df = pd.read_csv("./dataset/total_label_20240912.csv")
# df = df.drop(columns=['original'])

In [4]:
np_signal = np.array(signal)
print(np_signal.shape)

(2400, 5000, 12)


In [5]:
# resampling to 250Hz
def resampling(array, freq, kind='linear'):
    t = np.linspace(1, len(array), len(array))
    f = interpolate.interp1d(t, array, kind=kind)
    t_new = np.linspace(1, len(array), int(len(array)/freq * 250))
    new_array = f(t_new)
    return new_array


# standard normalization 
def normalize(data):
    scaler = StandardScaler()
    data_norm = scaler.fit_transform(data)
    return data_norm


# function of R peaks of a resampled trial
def R_Peaks(ecg_data):
    # get R Peak positions
    pos = []
    # get R Peak intervals
    trial_interval = []
    for ch in range(ecg_data.shape[1]):
        cleaned_ecg = nk.ecg_clean(ecg_data[:, ch], sampling_rate=250, method='neurokit')
        signals, info = nk.ecg_peaks(cleaned_ecg, sampling_rate=250, correct_artifacts=False)
        peaks = signals[signals['ECG_R_Peaks']==1].index.to_list()
        pos.append(peaks)
        channel_interval = []
        for i in range(len(peaks)-1):
            channel_interval.append(peaks[i+1] - peaks[i])
        trial_interval.append(channel_interval)
        
    df_peaks = pd.DataFrame(pos) # [C=15, num of the R-Peaks of a channel]
    df = pd.DataFrame(trial_interval).T
    med = df.median()
    return df, med, df_peaks

In [10]:
# # get median R-Peak intervals for all trials
# med_intervals = []
# li_abnormal_trial = []

# for ecg_data in tqdm(np_signal):
#     trial = []

#     for ch in range(ecg_data.shape[1]):
#         data = resampling(ecg_data[:, ch], freq=500, kind='linear')
#         trial.append(data)

#     trial = np.array(trial).T
#     trial_norm = normalize(trial)

#     _, med, _ = R_Peaks(trial_norm)
#     med_intervals.append(med.to_list())

# df_med_intervals = pd.DataFrame(med_intervals).T

100%|██████████| 2400/2400 [00:49<00:00, 48.21it/s]


In [6]:
cnt = 0

for ecg_data in tqdm(np_signal):
    sub = []
    trial = []
    for ch in range(ecg_data.shape[1]):
        # data = resampling(ecg_data[:,ch], freq=500, kind='linear')
        data = ecg_data[:, ch]
        trial.append(data)
    trial = np.array(trial).T
    # trial_norm = normalize(trial)
    sub.append(trial)

    sub = np.array(sub)
    # sub = sub.reshape(-1, 2500, sub.shape[-1])  # split 10s trial into 1s sample

    np.save('./dataset/KMedicon/Feature/feature_{}.npy'.format(cnt), sub)

    cnt +=1

print(sub.shape)

100%|██████████| 2400/2400 [00:00<00:00, 3323.36it/s]

(1, 5000, 12)


In [7]:
label = np.array(df['original'].to_list())
print(label.shape)

np.save("./dataset/KMedicon/Label/label.npy", label)

(2400,)


In [7]:
np.load('./dataset/KMedicon/Feature/feature_0.npy').shape

(1, 5000, 12)

In [8]:
lbl = np.load('./dataset/KMedicon/Label/label.npy')

In [9]:
list(np.where(lbl[:] == 0)[0])

[0,
 1,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 25,
 27,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 38,
 41,
 42,
 43,
 44,
 45,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 60,
 61,
 63,
 64,
 65,
 66,
 67,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 77,
 78,
 79,
 80,
 81,
 82,
 84,
 85,
 89,
 90,
 91,
 92,
 93,
 94,
 96,
 98,
 99,
 100,
 101,
 102,
 104,
 105,
 106,
 108,
 109,
 110,
 111,
 112,
 113,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 126,
 129,
 131,
 133,
 134,
 135,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 148,
 149,
 150,
 151,
 153,
 156,
 158,
 160,
 161,
 163,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 174,
 175,
 176,
 177,
 178,
 179,
 181,
 183,
 184,
 185,
 186,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 197,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 220,
 221,
 222,
 224,
 225,
 228,
 229,
 230,
 234,
 235,
 

In [11]:
lbl_lst = list(lbl[np.where(lbl[:] == 1)[0]])